# GraspGPT 模型推理与生成

本notebook用于从checkpoint加载GraspGPT模型，加载sequence序列数据，并进行token生成预测。

## 1. 导入必要的库和模块

In [ ]:
import os
import sys
import json
import time
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import deepspeed
from deepspeed import comm as dist

# 设置路径并导入graspGPT模块
current_dir = os.getcwd()
sys.path.append(current_dir)
sys.path.append(os.path.join(current_dir, 'graspGPT'))

from graspGPT.model.model import graspGPT
from graspGPT.model.utils import CfgNode as CN
from graspGPT.model.token_manager import get_token_manager, decode_sequence, encode_sequence
from graspGPT.model.parser_and_serializer import Serializer, Parser,Seq
import random

print("模块导入完成")

## 2. 配置参数

In [ ]:
# 设置参数
checkpoint_path = "output/checkpoints/allfullpoints"  # 修改为实际的checkpoint路径
sequence_file = "output/scene_0000_objects_merged_aligned_seq.pth"  # 修改为align_coords.py生成的pth文件路径
deepspeed_config_path = "deepspeed_config.json"  # DeepSpeed配置文件路径

# 生成参数
max_new_tokens = 2000
temperature = 0.3
do_sample = True
top_k = None
num_sequences = 1
seed = 42

# 设置随机种子
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

print(f"Checkpoint路径: {checkpoint_path}")
print(f"序列文件路径: {sequence_file}")
print(f"生成参数: max_new_tokens={max_new_tokens}, temperature={temperature}")

## 3. 辅助函数定义

In [ ]:
def parse_config_string(config_str):
    """解析字符串格式的配置到字典"""
    config_dict = {}
    for line in config_str.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip()
            value = value.strip()
            
            # 尝试解析值
            try:
                if value.lower() == 'true':
                    value = True
                elif value.lower() == 'false':
                    value = False
                elif value.lower() == 'none':
                    value = None
                elif value.startswith('(') and value.endswith(')'):
                    value = eval(value)
                elif value.startswith('[') and value.endswith(']'):
                    value = eval(value)
                elif '.' in value:
                    try:
                        value = float(value)
                    except ValueError:
                        pass
                else:
                    try:
                        value = int(value)
                    except ValueError:
                        pass
            except:
                pass
            
            config_dict[key] = value
    
    return config_dict

def load_training_config(checkpoint_dir):
    """从checkpoint目录加载训练配置"""
    # 尝试从training_state.json加载配置
    training_state_path = os.path.join(checkpoint_dir, 'training_state.json')
    if os.path.exists(training_state_path):
        with open(training_state_path, 'r') as f:
            training_state = json.load(f)
            if 'config' in training_state:
                config_data = training_state['config']
                
                if isinstance(config_data, dict):
                    parsed_config = {}
                    for section_name, section_value in config_data.items():
                        if isinstance(section_value, str):
                            parsed_config[section_name] = parse_config_string(section_value)
                        else:
                            parsed_config[section_name] = section_value
                    return CN.from_dict(parsed_config)
                else:
                    return CN.from_dict(config_data)
    
    # 备用：查找config.json文件
    search_dirs = [checkpoint_dir, os.path.dirname(checkpoint_dir)]
    config_names = ['config.json', 'training_config.json']
    
    for search_dir in search_dirs:
        for config_name in config_names:
            config_path = os.path.join(search_dir, config_name)
            if os.path.exists(config_path):
                with open(config_path, 'r') as f:
                    config_dict = json.load(f)
                return CN.from_dict(config_dict)
    
    raise FileNotFoundError(f"未找到配置文件在 {checkpoint_dir} 或其父目录中")

print("辅助函数定义完成")

## 4. 第一部分：从checkpoint目录加载模型和参数

In [ ]:
# 加载训练配置
print("正在加载训练配置...")
config = load_training_config(checkpoint_path)
print("训练配置加载成功")

# 打印模型配置信息
print(f"模型类型: {getattr(config.model, 'model_type', 'custom')}")
print(f"词汇大小: {config.model.vocab_size}")
print(f"块大小: {config.model.block_size}")

# 修复模型配置以满足XOR条件
if hasattr(config.model, 'model_type') and config.model.model_type:
    if hasattr(config.model, 'n_layer'):
        config.model.n_layer = None
    if hasattr(config.model, 'n_head'):
        config.model.n_head = None  
    if hasattr(config.model, 'n_embd'):
        config.model.n_embd = None

In [ ]:
# 获取token管理器和词汇表
print("正在获取token管理器...")
token_manager = get_token_manager()

# 生成token映射
img_h, img_w, img_d =80, 54, 34  # 默认体积维度
config.dataset.data_path = 'output/precomputed_data/'
# 尝试从数据中获取实际维度
if hasattr(config.dataset, 'data_path') and config.dataset.data_path:
    import glob
    data_files = glob.glob(os.path.join(config.dataset.data_path, "*.pth"))
    if data_files:
        sample_file = data_files[0]
        print(f"从样本文件获取体积维度: {os.path.basename(sample_file)}")
        raw_data = torch.load(sample_file, weights_only=False)
        if 'volume_dims' in raw_data:
            img_h, img_w, img_d = raw_data['volume_dims']
            print(f"从数据获取的体积维度: {img_h}x{img_w}x{img_d}")

token_mapping = token_manager.generate_mapping(img_h, img_w, img_d)
vocab_size = len(token_mapping)

# 更新配置中的词汇大小
config.model.vocab_size = vocab_size

print(f"Token管理器词汇大小: {vocab_size}")
print(f"使用的体积维度: {img_h}x{img_w}x{img_d}")

In [ ]:
# 创建模型
print("正在创建模型...")
model = graspGPT(config.model)
param_count = sum(p.numel() for p in model.parameters()) / 1e6
print(f"模型创建完成: {param_count:.2f}M 参数")

# 加载DeepSpeed配置
print("正在加载DeepSpeed配置...")
with open(deepspeed_config_path, 'r') as f:
    ds_config = json.load(f)

# 配置推理模式
ds_config.update({
    "train_batch_size": 1,
    "train_micro_batch_size_per_gpu": 1,
    "gradient_accumulation_steps": 1,
})

if ds_config.get("bf16", {}).get("enabled", False):
    print("bf16已启用用于推理")

print("DeepSpeed配置加载完成")

In [ ]:
# 加载checkpoint
print(f"正在加载checkpoint: {checkpoint_path}")

# 解析checkpoint路径
if os.path.isfile(checkpoint_path):
    parent_dir = os.path.dirname(checkpoint_path)
    tag = os.path.basename(checkpoint_path)
elif os.path.isdir(checkpoint_path):
    parent_dir = os.path.dirname(checkpoint_path)
    tag = os.path.basename(checkpoint_path)
else:
    raise ValueError(f"Checkpoint路径不存在: {checkpoint_path}")

# 尝试使用DeepSpeed加载
try:
    # 初始化DeepSpeed引擎用于推理
    model_engine, _, _, _ = deepspeed.initialize(
        model=model,
        config=ds_config,
        model_parameters=model.parameters()
    )
    
    # 加载checkpoint
    _, client_state = model_engine.load_checkpoint(parent_dir, tag=tag)
    print(f"使用DeepSpeed成功加载checkpoint")
    
except Exception as e:
    print(f"DeepSpeed加载失败: {e}")
    print("回退到PyTorch加载...")
    
    # 回退：使用常规PyTorch加载
    checkpoint_file = os.path.join(checkpoint_path, 'mp_rank_00_model_states.pt')
    if os.path.exists(checkpoint_file):
        print(f"使用PyTorch从以下位置加载checkpoint: {checkpoint_file}")
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        checkpoint = torch.load(checkpoint_file, map_location=device)
        
        state_dict = checkpoint.get('module', checkpoint)
        model.load_state_dict(state_dict, strict=False)
        
        if torch.cuda.is_available():
            model = model.cuda()
        
        print(f"使用PyTorch成功加载checkpoint")
        
        # 创建模型包装器以兼容DeepSpeed接口
        class ModelWrapper:
            def __init__(self, model):
                self.module = model
                self.model = model
                self.local_rank = 0
                
            def eval(self):
                self.module.eval()
                
            def __call__(self, *args, **kwargs):
                return self.module(*args, **kwargs)
        
        model_engine = ModelWrapper(model)
    else:
        raise ValueError(f"找不到checkpoint文件: {checkpoint_file}")

print("模型加载完成！")

## 5. 第二部分：从外部pth文件加载sequence序列

In [ ]:
# 加载序列数据
print(f"正在加载序列数据: {sequence_file}")

if not os.path.exists(sequence_file):
    raise FileNotFoundError(f"序列文件不存在: {sequence_file}")

# 加载序列数据
sequence_data = torch.load(sequence_file, weights_only=False)

# 检查数据结构
print(f"序列数据键: {list(sequence_data.keys()) if isinstance(sequence_data, dict) else 'not a dict'}")

# 提取token序列
if isinstance(sequence_data, dict):
    if 'tokens' in sequence_data:
        token_sequence = sequence_data['tokens']
    elif 'sequence' in sequence_data:
        token_sequence = sequence_data['sequence']
    elif 'token_ids' in sequence_data:
        token_sequence = sequence_data['token_ids']
    else:
        # 假设第一个可用的键包含序列
        key = list(sequence_data.keys())[0]
        token_sequence = sequence_data[key]
        print(f"使用键 '{key}' 作为token序列")
elif isinstance(sequence_data, (list, tuple)):
    token_sequence = sequence_data
else:
    token_sequence = sequence_data



# --------------(training data

In [ ]:
sequence_file = 'output/precomputed_data/precomputed_batch_87750_11.pth'
# 加载序列数据
print(f"正在加载序列数据: {sequence_file}")

if not os.path.exists(sequence_file):
    raise FileNotFoundError(f"序列文件不存在: {sequence_file}")

# 加载序列数据
sequence_data = torch.load(sequence_file, weights_only=False)

sequence_data = sequence_data[5]['raw_tokens']

# 检查数据结构
print(f"序列数据键: {list(sequence_data.keys()) if isinstance(sequence_data, dict) else 'not a dict'}")


print(f"使用键 raw_tokens 作为token序列")

token_sequence = decode_sequence(sequence_data, token_mapping)
parser = Parser(token_sequence)
token_sequence = parser.parse()
print(len(token_sequence.items))


## 构造场景数据

In [ ]:
sbs= token_sequence.items
#random.shuffle(sbs)
New_seq = Seq(items=sbs[:4])
print(sbs[0].tag, sbs[1].tag)
flat_tokens = Serializer.serialize(New_seq)

In [ ]:
scene_promt = encode_sequence(flat_tokens, token_mapping)[:-1]
print(len(scene_promt),scene_promt[-10:])

## 6. 第三部分：在序列尾部加入新的指定tokens作为prompt，让model接下去预测

In [ ]:
# 指定要添加到序列尾部的tokens作为prompt
# 这里可以根据需要修改，例如添加特定的命令tokens
additional_tokens = [
    token_mapping['detectgrasp'],  # 检测抓取命令
    token_mapping['grasp'],         # 抓取命令
    #token_mapping['object24']
]

print(f"要添加的tokens: {additional_tokens}")
print(f"对应的token名称: {[k for k, v in token_mapping.items() if v in additional_tokens]}")

# 将新的tokens添加到序列尾部
prompt_sequence = scene_promt + additional_tokens
#prompt_sequence = scene_promt

print(f"添加prompt后的序列长度: {len(prompt_sequence)}")
print(f"完整prompt序列的最后20个tokens: {prompt_sequence[-20:]}")

In [ ]:
# 准备输入张量
def prepare_input_from_tokens(token_ids, max_length=None):
    """从token ID列表准备输入"""
    if not token_ids:
        raise ValueError("提供的token ID列表为空")
    
    # 转换为张量
    input_ids = torch.tensor([token_ids], dtype=torch.long)
    
    # 如果需要，进行截断
    if max_length and input_ids.size(1) > max_length:
        input_ids = input_ids[:, -max_length:]
    
    return input_ids

# 准备输入
input_ids = prepare_input_from_tokens(prompt_sequence, config.model.block_size)
input_ids = input_ids.repeat(2,1)
original_length = input_ids.size(1)

print(f"输入张量形状: {input_ids.shape}")
print(f"原始序列长度: {original_length}")

In [ ]:
# 配置生成参数
generation_config = {
    'max_new_tokens': max_new_tokens,
    'temperature': temperature,
    'do_sample': do_sample,
    'top_k': top_k,
    'eos_token_id': token_mapping.get('end', None)
}

print(f"生成配置: {generation_config}")

In [ ]:
# 执行生成
def generate_with_model(model_engine, input_ids, generation_config):
    """使用模型生成序列"""
    # 将输入移动到正确的设备
    if hasattr(model_engine, 'local_rank') and model_engine.local_rank is not None:
        device = f'cuda:{model_engine.local_rank}'
    else:
        # 回退：从模型参数获取设备
        device = next(model_engine.module.parameters()).device
    
    input_ids = input_ids.to(device)
    
    # 将模型设置为评估模式
    model_engine.eval()
    
    with torch.no_grad():
        # 生成序列
        if hasattr(model_engine.module, 'generate'):
            generated = model_engine.module.generate(
                idx=input_ids,
                max_new_tokens=generation_config.get('max_new_tokens', 50),
                temperature=generation_config.get('temperature', 1.0),
                do_sample=generation_config.get('do_sample', True),
                top_k=generation_config.get('top_k', None),
                end_token=generation_config.get('eos_token_id', None)
            )
        else:
            raise ValueError("模型没有generate方法")
    
    return generated

print("开始生成...")
start_time = time.time()

print("输入长度:",input_ids.size())
# 执行生成
generated = generate_with_model(model_engine, input_ids, generation_config)

generation_time = time.time() - start_time

print(f"生成完成，耗时: {generation_time:.2f}秒")
print(f"生成的序列长度: {generated.size(1)} tokens. {generated.size()}")

In [ ]:
from extract_sample_and_export import visualize_tokens

all_scene  = generated.squeeze().detach().cpu()
all_scene[:,-1] = 98
for i in range(all_scene.size(0)):
    visualize_tokens(all_scene[i], token_mapping, volume_dims =(img_h, img_w, img_d), bbox_min = np.array([-0.3, -0.2, 0]), voxel_size = 0.0075, output_dir = f'./output/tokens_visual/{i}')

In [ ]:
print(all_scene[1][:].cpu().numpy().tolist())

In [ ]:
# 完整序列（原始 + prompt + 生成）
full_sequence = input_ids[0].cpu().numpy().tolist() + decoded_tokens

print(f"完整序列长度: {len(full_sequence)}")
print(f"完整序列的最后30个tokens: {full_sequence[-30:]}")

# 保存结果
result = {
    'original_sequence_length': len(token_sequence),
    'prompt_tokens': additional_tokens,
    'input_length': original_length,
    'generated_tokens': decoded_tokens,
    'generated_token_names': decoded_names,
    'total_length': len(full_sequence),
    'generation_time': generation_time,
    'generation_config': generation_config,
    'full_sequence': full_sequence
}

print("\n=== 生成结果总结 ===")
print(f"原始序列长度: {result['original_sequence_length']}")
print(f"添加的prompt tokens: {result['prompt_tokens']}")
print(f"输入长度: {result['input_length']}")
print(f"生成的新tokens: {result['generated_tokens']}")
print(f"生成时间: {result['generation_time']:.2f}秒")

## 7. 可选：可视化生成结果

In [ ]:
# 可选：使用可视化功能（如果extract_sample_and_export模块可用）
try:
    sys.path.append('../')
    from extract_sample_and_export import visualize_tokens
    
    # 设置可视化参数（根据实际情况调整）
    volume_dims = (img_h, img_w, img_d)
    bbox_min = np.array([-0.3, -0.2, 0])  # 根据实际情况调整
    voxel_size = 0.0075  # 根据实际情况调整
    
    # 可视化完整序列
    output_dir = "./output/generation_visual/notebook_result"
    
    visualize_tokens(
        tokens=full_sequence,
        token_mapping=token_mapping,
        volume_dims=volume_dims,
        bbox_min=bbox_min,
        voxel_size=voxel_size,
        output_dir=output_dir
    )
    
    print(f"可视化结果保存到: {output_dir}")
    
except ImportError:
    print("可视化模块不可用，跳过可视化步骤")
except Exception as e:
    print(f"可视化过程中出错: {e}")

## 8. 保存结果到文件

In [ ]:
# 保存结果到JSON文件
output_file = "generation_result.json"
with open(output_file, 'w') as f:
    json.dump(result, f, indent=2)

print(f"结果已保存到: {output_file}")

# 同时保存为.pth文件以便后续使用
torch.save({
    'generated_sequence': full_sequence,
    'metadata': result
}, "generation_result.pth")

print("结果也已保存为generation_result.pth")
print("\n生成完成！")